## Necessary Libraries

In [1]:
## pyb stuff
import pybaseball as pyb
from pybaseball import  playerid_lookup
from pybaseball import statcast_pitcher
from pybaseball import pitching_stats
from pybaseball import batting_stats
from pybaseball import schedule_and_record

import pandas as pd 
pd.set_option('display.max_columns', None)

import numpy as np

## added yesterday's date so the data we get is updated everytime the statcast data cell is ran
from datetime import date, timedelta, datetime
today = date.today()
yesterday = today - timedelta(days=1)
formatted_date = yesterday.strftime("%Y-%m-%d")

import warnings
warnings.filterwarnings("ignore")

## Defining Functions

In [4]:
start_date = '2024-03-01'
end_date = '2025-08-15'

statcast_data = pyb.statcast(start_date, end_date)

statcast_data = statcast_data.sort_values(by='game_date').reset_index(drop=True)
statcast_data = statcast_data[~statcast_data['pitch_type'].isna()].sort_values(by='game_date').reset_index(drop=True) ## gets rid of the null pitches
statcast_data.head()

This is a large query, it may take a moment to complete
Skipping offseason dates
Skipping offseason dates


100%|██████████| 400/400 [01:21<00:00,  4.93it/s]


,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,spin_dir,spin_rate_deprecated,break_angle_deprecated,break_length_deprecated,zone,des,game_type,stand,p_throws,home_team,away_team,type,hit_location,bb_type,balls,strikes,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,inning_topbot,hc_x,hc_y,tfs_deprecated,tfs_zulu_deprecated,umpire,sv_id,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,fielder_2,fielder_3,fielder_4,fielder_5,fielder_6,fielder_7,fielder_8,fielder_9,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,pitch_name,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp,bat_speed,swing_length,estimated_slg_using_speedangle,delta_pitcher_run_exp,hyper_speed,home_score_diff,bat_score_diff,home_win_exp,bat_win_exp,age_pit_legacy,age_bat_legacy,age_pit,age_bat,n_thruorder_pitcher,n_priorpa_thisgame_player_at_bat,pitcher_days_since_prev_game,batter_days_since_prev_game,pitcher_days_until_next_game,batter_days_until_next_game,api_break_z_with_gravity,api_break_x_arm,api_break_x_batter_in,arm_angle,attack_angle,attack_direction,swing_path_tilt,intercept_ball_minus_batter_pos_x_inches,intercept_ball_minus_batter_pos_y_inches
0,FF,2024-03-15,93.8,-1.45,6.17,"Houser, Adrian",608336,605288,NaN,foul,<NA>,<NA>,<NA>,<NA>,11,Joey Gallo pops out to catcher Francisco Alvar...,S,L,R,NYM,WSH,S,<NA>,NaN,0,0,2024,-0.76,1.26,-0.88,2.99,<NA>,<NA>,<NA>,2,1,Top,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,3.157271,-136.531806,-5.310178,-10.239995,29.364474,-15.430712,3.43,1.73,258,80.4,54,94.3,2275,6.5,747961,682626,668901,621563,683146,596019,621466,664056,516782,53.99,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,4,1,4-Seam Fastball,0,1,1,0,1,0,1,0,Infield shade,Standard,222,0.0,-0.041,<NA>,<NA>,<NA>,0.041,88.0,-1,1,0.433,0.567,31,30,31,31,1,0,<NA>,<NA>,<NA>,<NA>,1.32,0.76,-0.76,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,FF,2024-03-15,94.4,-2.21,6.03,"Smith-Shawver, AJ",608348,700363,field_out,hit_into_play,<NA>,<NA>,<NA>,<NA>,8,Carson Kelly flies out to right fielder Eli Wh...,S,R,R,DET,ATL,X,9,fly_ball,1,0,2024,-0.85,1.65,-0.17,1.68,<NA>,682985,<NA>,1,3,Bot,193.19,114.21,<NA>,<NA>,<NA>,<NA>,7.28227,-137.035703,-9.388771,-12.290337,30.408892,-9.464813,3.28,1.61,268,91.1,47,95.3,2118,6.9,747956,644433,621566,645277,641645,606115,672284,671739,642201,53.56,0.003,<NA>,0.0,<NA>,0,0,3,24,2,4-Seam Fastball,4,0,4,0,0,4,4,0,Standard,Standard,212,-0.011,-0.289,<NA>,<NA>,0.006,0.289,91.1,4,4,0.909,0.909,21,29,22,30,2,1,<NA>,<NA>,<NA>,<NA>,0.92,0.85,0.85,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2,SL,2024-03-15,86.5,-2.46,6.08,"Smith-Shawver, AJ",570482,700363,NaN,swinging_strike,<NA>,<NA>,<NA>,<NA>,5,Gio Urshela singles on a line drive to center ...,S,R,R,DET,ATL,S,<NA>,NaN,1,0,2024,0.21,0.54,-0.23,2.86,678009,<NA>,<NA>,0,3,Bot,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,4.94394,-126.116968,-2.745282,1.454022,19.936556,-26.01901,3.46,1.68,<NA>,<NA>,<NA>,88.2,2120,6.7,747956,644433,621566,645277,641645,606115,672284,671739,642201,53.78,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,21,2,Slider,2,0,2,0,0,2,2,0,Strategic,Standard,220,0.0,-0.093,<NA>,<NA>,<NA>,0.093,<NA>,2,2,0.836,0.836,21,32,22,33,2,1,<NA>,<NA>,<NA>,<NA>,2.44,-0.21,-0.21,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3,CU,2024-03-15,74.1,-2.17,6.06,"Smith-Shawver, AJ",570482,700363,NaN,ball,<NA>,<NA>,<NA>,<NA>,11,Gio Urshela singles on a line drive to center ...,S,R,R,DET,ATL,B,<NA>,NaN,1,1,2024,1.02,-1.06,-0.91,3.55,678009,<NA>,<NA>,0,3,Bot,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0.800755,-107.839599,4.028733,7.814088,18.8554,-41.388552,3.46,1.68,<NA>,<NA>,<NA>,74.8,2246,6.8,747956,644433,621566,645277,641645,606115,672284,671739,642201,53.65,<NA>,<NA>,<NA>,<NA>,<N

In [5]:
mlb_pitchers = statcast_data.groupby(['pitcher', 'pitch_name']).agg(pitcher_name = ('player_name', 'max'),
                                                                     handedness = ('p_throws', 'max'),
                                                                     count_pitch = ('player_name', 'count'),
                                                                     spin_rate = ('release_spin_rate', 'mean'),
                                                                     release_speed = ('release_speed', 'mean'), 
                                                                     horz_break = ('pfx_x', 'mean'), 
                                                                     vert_break = ('pfx_z', 'mean'), 
                                                                     extension = ('release_extension', 'mean'), 
                                                                     spin_axis = ('spin_axis', 'mean')
                                                                     ).reset_index()

In [6]:
mlb_pitchers.head()

,pitcher,pitch_name,pitcher_name,handedness,count_pitch,spin_rate,release_speed,horz_break,vert_break,extension,spin_axis
0,434378,4-Seam Fastball,"Verlander, Justin",R,1610,2414.603106,93.849752,-0.732888,1.573752,6.017888,206.999379
1,434378,Changeup,"Verlander, Justin",R,314,1751.605096,84.502229,-1.113567,0.782675,6.043949,229.531847
2,434378,Curveball,"Verlander, Justin",R,609,2710.807882,78.127586,0.613383,-1.067028,6.000657,38.284072
3,434378,Sinker,"Verlander, Justin",R,10,2449.3,92.88,-1.004,1.345,5.99,198.7
4,434378,Slider,"Verlander, Justin",R,734,2468.516349,87.233379,0.324755,0.494223,6.047684,171.158038


In [7]:
total_pitches = mlb_pitchers.groupby(['pitcher_name'])[['count_pitch']].sum().reset_index()
total_pitches = total_pitches.rename(columns = {'count_pitch': 'total_pitches'})
mlb_pitchers_master = mlb_pitchers.merge(total_pitches, on = ['pitcher_name'], how = 'left')
mlb_pitchers_master['usage%'] = round((mlb_pitchers_master['count_pitch'] / mlb_pitchers_master['total_pitches']) * 100, 1)
mlb_pitchers_master

,pitcher,pitch_name,pitcher_name,handedness,count_pitch,spin_rate,release_speed,horz_break,vert_break,extension,spin_axis,total_pitches,usage%
0,434378,4-Seam Fastball,"Verlander, Justin",R,1610,2414.603106,93.849752,-0.732888,1.573752,6.017888,206.999379,3389,47.5
1,434378,Changeup,"Verlander, Justin",R,314,1751.605096,84.502229,-1.113567,0.782675,6.043949,229.531847,3389,9.3
2,434378,Curveball,"Verlander, Justin",R,609,2710.807882,78.127586,0.613383,-1.067028,6.000657,38.284072,3389,18.0
3,434378,Sinker,"Verlander, Justin",R,10,2449.3,92.88,-1.004,1.345,5.99,198.7,3389,0.3
4,434378,Slider,"Verlander, Justin",R,734,2468.516349,87.233379,0.324755,0.494223,6.047684,171.158038,3389,21.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6149,829272,Changeup,"Ogasawara, Shinnosuke",L,51,1609.882353,79.919608,1.22902,0.637451,6.166667,126.686275,255,20.0
6150,829272,Knuckle Curve,"Ogasawara, Shinnosuke",L,40,2718.325,71.5475,-0.61275,-1.6745,6.0075,342.375,255,15.7
6151,829272,Sinker,"Ogasawara, Shinnosuke",L,3,2028.0,86.166667,1.216667,1.11,6.233333,136.666667,255,1.2
6152,829272,Slider,"Ogasawara, Shinnosuke",L,54,2431.740741,80.972222,-0.210185,0.182222,6.175926,225.759259,255,21.2


In [ ]:
trackman = pd.read_csv('../../Files/merged_trackman_games.csv')